In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)

#from pandas import DataFrame, read_csv
import pandas as pd

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

import sys  
#sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#from AdaBandFlt import *
from V2AdaBandFlt import *

%matplotlib tk

In [3]:
#load data
# file path of csv file
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [4]:
#load data
# file path of csv file

Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2_tstamp.csv'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2_tstamp.csv'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbase_isi.csv'

# create dataframe
df_ = pd.read_csv(Location, sep=',')

In [5]:
df.head()

,El 21,El 31,El 41,El 22,El 32,El 42,El 23,El 33,El 43,El 15
%t,,,,,,,,,,
0.00,-2.61,1.02,0.80,1.93,-1.25,-1.59,-0.57,4.09,-1.14,-0.34
0.04,-3.30,0.00,2.05,1.25,-1.02,-0.45,-1.82,2.61,-0.45,-0.80
0.08,-2.73,0.11,2.73,1.48,0.45,1.14,-1.14,3.52,-0.23,-0.80
0.12,-3.07,-0.45,1.25,0.80,-0.57,-0.57,-1.25,2.50,-1.02,-0.57
0.16,-2.39,1.48,0.34,1.14,0.00,-0.57,-0.34,1.48,-0.23,0.45


In [6]:
df_.head()

,T[s],sppa0001_21_Timestamp_Unit0,T[s].1,sppa0001_31_Timestamp_Unit0,T[s].2,sppa0001_41_Timestamp_Unit0,T[s].3,sppa0001_22_Timestamp_Unit0,T[s].4,sppa0001_32_Timestamp_Unit0,T[s].5,sppa0001_42_Timestamp_Unit0,T[s].6,sppa0001_23_Timestamp_Unit0,T[s].7,sppa0001_33_Timestamp_Unit0,T[s].8,sppa0001_43_Timestamp_Unit0,T[s].9,sppa0001_15_Timestamp_Unit0
0,1.63776,NaN,0.38532,NaN,0.85184,NaN,2.14396,NaN,2.13620,NaN,0.56064,NaN,2.14936,NaN,0.47808,NaN,2.13872,NaN,7.96416,NaN
1,1.99384,NaN,1.08808,NaN,2.12424,NaN,2.15492,NaN,2.14376,NaN,2.14380,NaN,2.15504,NaN,0.90124,NaN,2.14168,NaN,16.50048,NaN
2,2.13476,NaN,2.11608,NaN,2.13104,NaN,2.16148,NaN,2.15196,NaN,2.15204,NaN,2.16224,NaN,2.11860,NaN,2.14432,NaN,17.73908,NaN
3,2.13824,NaN,2.12060,NaN,2.13352,NaN,2.16732,NaN,2.15752,NaN,2.15772,NaN,2.17080,NaN,2.14616,NaN,2.15276,NaN,NaN,NaN
4,2.14420,NaN,2.13140,NaN,2.13628,NaN,2.17520,NaN,2.16072,NaN,2.16072,NaN,2.17296,NaN,2.14848,NaN,2.15540,NaN,NaN,NaN


In [51]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

#xminnoise = int(np.round(11114*(fs/1000)))
#xmaxnoise = int(np.round(18511*(fs/1000)))

#noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

#xminspike = int(np.round(130826*(fs/1000)))
#xmaxspike = int(np.round(131699*(fs/1000)))

#burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

#begin_data = signal_filtered.iloc[:500000]

In [52]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [53]:
fs = 25000
align_method = 'indice_1er_depass'

In [10]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

"""plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')"""

"plt.figure()\nplt.plot(noise_levels)\nplt.grid(True)\nplt.xlabel('Time')\nplt.ylabel('Noise Amplitude [µV]')\nplt.title('Noise Levels')"

In [100]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0012, 
                           noise_window_size = 0.01,
                           threshold_factor = 2.5,
                           maxseparation = 0.001)  
                
#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes
0,1149,1149,nan,1163,14.339498
1,9155,9155,nan,9164,11.351349
2,12001,12001,nan,12010,13.830043
3,14975,14973,nan,14994,12.571033
4,16241,16239,nan,16259,12.468565
...,...,...,...,...,...
622,509307,509305,nan,509326,11.131265
623,511259,511258,nan,511281,11.286355
624,512271,512268,nan,512281,12.000984
625,516878,516877,nan,516902,11.347936


In [120]:
signal = signal_filtered

In [102]:
toto = df_.iloc[:, 2].dropna().values
arr = toto*fs
spike_info_ = pd.DataFrame(arr.astype('int') , columns = [align_method])


In [103]:
signal = full_signal

In [104]:
spike_data = record_spikes(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_ = record_spikes(signal, fs, spike_info_,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

(628, 76)
(466, 76)


In [105]:
def print_spikes(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = fs,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.index-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()

# Comparaison spikes

In [106]:
sophie_spike = spike_info_[align_method].values
our_spike = spike_info[align_method].values
temporal_tol = 0.0006

i = 0
j = 0

joint_spikes_sophie_side = []
solo_spikes_sophie_side = []
joint_spikes_our_side = []
solo_spikes_our_side = []
tol = int(np.round(fs * temporal_tol))

while(True):
    if sophie_spike[i] < our_spike[j] - tol:
        solo_spikes_sophie_side.append(i)
        i += 1
    elif sophie_spike[i] > our_spike[j] + tol:
        solo_spikes_our_side.append(j)
        j += 1
    else:
        joint_spikes_sophie_side.append(i)
        joint_spikes_our_side.append(j)
        i += 1
        j += 1
    if(i == len(sophie_spike)-1 or j == len(our_spike)-1):
        break
    
print('Spikes detected by sophie: %d' %len(sophie_spike))
print('Spikes detected by our AdabandFlt: %d' %len(our_spike))
print(" ")
print('joint_spikes_sophie_side')
print('Spikes detected by Sophie that we have in common: %d' %len(joint_spikes_sophie_side))
print(joint_spikes_sophie_side)
print(" ")
print('solo_spikes_sophie_side')
print('Spikes detected by sophie only: %d' %len(solo_spikes_sophie_side))
print(solo_spikes_sophie_side)
print(" ")
print('joint_spikes_our_side')
print('Spikes detected by us that we have in common: %d' %len(joint_spikes_our_side))
print(joint_spikes_our_side)
print('solo_spikes_our_side')
print('Spikes detected by us only: %d' %len(solo_spikes_our_side))
print(solo_spikes_our_side)

Spikes detected by sophie: 465
Spikes detected by our AdabandFlt: 627
 
joint_spikes_sophie_side
Spikes detected by Sophie that we have in common: 189
[5, 14, 15, 17, 20, 21, 22, 23, 28, 29, 34, 35, 36, 38, 40, 44, 45, 49, 50, 53, 54, 55, 58, 59, 60, 65, 66, 71, 84, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 102, 103, 104, 105, 106, 108, 110, 115, 119, 120, 123, 124, 125, 127, 128, 129, 130, 131, 132, 137, 138, 139, 140, 141, 143, 145, 148, 150, 154, 155, 156, 159, 162, 163, 165, 166, 167, 171, 174, 178, 179, 181, 183, 184, 186, 187, 188, 191, 192, 195, 196, 197, 202, 204, 205, 208, 211, 213, 215, 221, 225, 228, 229, 230, 231, 235, 241, 242, 245, 246, 247, 251, 252, 253, 254, 255, 256, 266, 271, 272, 273, 276, 278, 280, 282, 284, 285, 286, 287, 299, 301, 303, 308, 315, 321, 327, 329, 331, 334, 335, 342, 348, 351, 354, 355, 359, 361, 364, 369, 370, 371, 373, 376, 380, 386, 387, 388, 396, 400, 404, 405, 407, 409, 411, 412, 416, 417, 418, 419, 422, 425, 429, 430, 431, 436, 437, 440, 

In [107]:
print("Petit Bilan: ")
print(" ")
print('Spikes detected by sophie: %d' %len(sophie_spike))
print('Spikes detected by our AdabandFlt: %d' %len(our_spike))
print(" ")
print('Spikes detected by Sophie that we have in common: %d' %len(joint_spikes_sophie_side))
print(" ")
print('Spikes detected by sophie only: %d' %len(solo_spikes_sophie_side))
print(" ")
print('Spikes detected by us only: %d' %len(solo_spikes_our_side))

Petit Bilan: 
 
Spikes detected by sophie: 465
Spikes detected by our AdabandFlt: 627
 
Spikes detected by Sophie that we have in common: 189
 
Spikes detected by sophie only: 275
 
Spikes detected by us only: 428


## Affichage oneline

In [108]:
spike_data_oneline_joint_our_side = record_spikes_oneline(signal, fs, spike_info.iloc[joint_spikes_our_side],
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_oneline_solo_our_side = record_spikes_oneline(signal, fs, spike_info.iloc[solo_spikes_our_side],
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

In [109]:
figure = plt.figure()
axes = figure.add_subplot(2, 1, 1)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_joint_our_side.index, spike_data_oneline_joint_our_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_joint_our_side')
axes.grid(True)
axes = figure.add_subplot(2, 1, 2)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_solo_our_side.index, spike_data_oneline_solo_our_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_solo_our_side')
axes.grid(True)

In [110]:
spike_data_oneline_joint_sophie_side = record_spikes_oneline(signal, fs, spike_info_.iloc[joint_spikes_sophie_side],
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_oneline_solo_sophie_side = record_spikes_oneline(signal, fs, spike_info_.iloc[solo_spikes_sophie_side],
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002)

In [111]:
figure = plt.figure()
axes = figure.add_subplot(2, 1, 1)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_joint_sophie_side.index, spike_data_oneline_joint_sophie_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_joint_sophie_side')
axes.grid(True)
axes = figure.add_subplot(2, 1, 2)
axes.plot(signal.index, signal.values)
axes.plot(spike_data_oneline_solo_sophie_side.index, spike_data_oneline_solo_sophie_side.values)
axes.set_xlabel('Time in ms')
axes.set_title('spike_data_oneline_solo_sophie_side')
axes.grid(True)

## Affichage spikes sophie only

In [112]:
print_spikes(spike_data_,
                 t_before_alignement = 0.001,
                 randomize = True,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60)

In [113]:
print_spikes(spike_data_,
                 t_before_alignement = 0.001,
                 y_lim_min = -50,
                 y_lim_max = 60)

## Affichages séparés

In [114]:
print_spikes(spike_data.iloc[:,joint_spikes_our_side],
                 t_before_alignement = 0.001,
                 randomize = True,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60)

In [115]:
print_spikes(spike_data.iloc[:,solo_spikes_our_side],
                 t_before_alignement = 0.001,
                 randomize = True,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60)

In [116]:
print_spikes(spike_data_.iloc[:,joint_spikes_sophie_side],
                 t_before_alignement = 0.001,
                 randomize = True,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60)

In [117]:
print_spikes(spike_data_.iloc[:,solo_spikes_sophie_side],
                 t_before_alignement = 0.001,
                 randomize = True,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60)

## Affichage barres

In [121]:
signal = signal_filtered

plt.figure()
plt.plot(signal.index, signal, 'b')

spike_info.iloc[joint_spikes_our_side][align_method]

for value in spike_info.iloc[joint_spikes_our_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.5, ymax=0.9, color = 'purple')
for value in spike_info_.iloc[joint_spikes_sophie_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.1, ymax=0.5, color = 'purple')
for value in spike_info.iloc[solo_spikes_our_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.5, ymax=0.9, color = 'g')
for value in spike_info_.iloc[solo_spikes_sophie_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.1, ymax=0.5, color = 'r')

plt.grid()
plt.show()

## Comparaison nos spikes + barre ou il y a indice logiciel

In [119]:
#plt.figure()
#plt.plot(signal.index, signal, 'b')

spike_data_onelinee = record_spikes_oneline(signal_filtered, fs, spike_info,
                  align_method,
                  t_before = 0.002,
                  t_after = 0.002)


figure = plt.figure()
axes = figure.add_subplot(1, 1, 1)
axes.plot(signal_filtered.index, signal_filtered.values)
axes.plot(spike_data_onelinee.index, spike_data_onelinee.values)
for value in spike_info_.iloc[joint_spikes_sophie_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.1, ymax=0.5, color = 'purple')
for value in spike_info_.iloc[solo_spikes_sophie_side][align_method].values/fs*1000:
    plt.axvline(x=value, ymin=0.1, ymax=0.5, color = 'purple')
axes.set_xlabel('Time in ms')
axes.grid(True)